In [1]:
import pandas as pd #data manipulation and analysis
import numpy as np #data arrays
from datetime import datetime #datetime
from datetime import time
from datetime import timedelta
import datetime
import glob #join csv's together
import os # makes file join OS independent
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px #data visualisation
from geopy import distance # Vincenty formula for latitude and longitude distance calculations

In [2]:
#load downloaded csv in, change date strings to datetime64 (date/time format)
#define variables that we will use for concatenating
path =r'C:\Users\zaydm\Documents\Repos\Google_Data_Analytics_Capstone\CSV'
file_type = 'csv'
seperator =','

#define our dataframe. Change date strings to datetime64 
#'infer_datetime_format' infers the format of the datetime strings - in this case parsing speed = 10x faster
bike_share = pd.concat([pd.read_csv(f, sep=seperator,parse_dates=['started_at', 'ended_at'],infer_datetime_format=True, dayfirst=True) 

#using glob/for loop to find the csv files from the specified folder to join them together
for f in glob.glob(os.path.join(path + "/*."+file_type))],ignore_index=True)

#rename datetime columns so they're better defined
bike_share.rename(columns= {'started_at' : 'start_time','ended_at' : 'end_time' }, inplace=True)

In [3]:
bike_share.head()

,ride_id,rideable_type,start_time,end_time,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,6C992BD37A98A63F,classic_bike,2021-04-12 18:25:36,2021-04-12 18:56:55,State St & Pearson St,TA1307000061,Southport Ave & Waveland Ave,13235,41.897448,-87.628722,41.948150,-87.663940,member
1,1E0145613A209000,docked_bike,2021-04-27 17:27:11,2021-04-27 18:31:29,Dorchester Ave & 49th St,KA1503000069,Dorchester Ave & 49th St,KA1503000069,41.805772,-87.592464,41.805772,-87.592464,casual
2,E498E15508A80BAD,docked_bike,2021-04-03 12:42:45,2021-04-07 11:40:24,Loomis Blvd & 84th St,20121,Loomis Blvd & 84th St,20121,41.741487,-87.658410,41.741487,-87.658410,casual
3,1887262AD101C604,classic_bike,2021-04-17 09:17:42,2021-04-17 09:42:48,Honore St & Division St,TA1305000034,Southport Ave & Waveland Ave,13235,41.903119,-87.673935,41.948150,-87.663940,member
4,C123548CAB2A32A5,docked_bike,2021-04-03 12:42:25,2021-04-03 14:13:42,Loomis Blvd & 84th St,20121,Loomis Blvd & 84th St,20121,41.741487,-87.658410,41.741487,-87.658410,casual


In [4]:
#The dataframe includes a few hundred entries when bikes were taken out of docks and checked for quality. These are not needed
bike_share.drop(index=bike_share[bike_share['rideable_type'] =='docked_bike'].index, inplace=True)

bike_share['rideable_type'].unique()

array(['classic_bike', 'electric_bike'], dtype=object)

In [5]:
missing_values_count = bike_share.isnull().sum()

missing_values_count[:]
# We can see from start_station_id there are 745376 missing values and from end_station_id there are 795782 missing values. 
# However these still include ride start/end times so we can keep them. 
# The data with end_lat| end_lng (4251) missing values will need to be removed (as using lat/lng for distance calculations)

ride_id                    0
rideable_type              0
start_time                 0
end_time                   0
start_station_name    745376
start_station_id      745373
end_station_name      795782
end_station_id        795782
start_lat                  0
start_lng                  0
end_lat                 4251
end_lng                 4251
member_casual              0
dtype: int64

In [6]:
bike_share.dropna(subset=['end_lat','end_lng'], inplace=True)

In [7]:
#create ride length column in minutes - needed for analysis
bike_share['ride_length_mins'] = (bike_share['end_time'] - bike_share['start_time'])/pd.Timedelta(minutes = 1)

In [8]:
#checking for any negative ride lengths as previous year data had some errors
print(bike_share[bike_share['ride_length_mins'] < 0])

                  ride_id  rideable_type          start_time  \
22361    BC53ECCBC76278FD   classic_bike 2021-04-07 16:11:33   
31844    209C097828F9CD43  electric_bike 2021-04-27 17:13:44   
292566   6E81034B446FC2FD  electric_bike 2021-04-23 09:43:39   
292678   318DD838369AEA61   classic_bike 2021-04-30 10:56:32   
293033   8ADD13BD8F6A7567   classic_bike 2021-04-17 12:43:36   
...                   ...            ...                 ...   
4969725  5AA2BC364BC7A569  electric_bike 2021-11-07 01:59:53   
4971172  F4E4485BFB33D916  electric_bike 2021-11-07 01:57:53   
4972542  B506DCD44974C575  electric_bike 2021-11-07 01:53:34   
5623702  2D97E3C98E165D80   classic_bike 2022-03-05 11:00:57   
5626879  7407049C5D89A13D  electric_bike 2022-03-05 11:38:04   

                   end_time                      start_station_name  \
22361   2021-04-07 16:11:26                 Ashland Ave & Grand Ave   
31844   2021-04-27 17:11:32                                     NaN   
292566  2021-04-23

In [9]:
#drop rows where columns have negative ride lengths (some bikes were taken out of circulation/ QC issues thus need to be deleted)
bike_share.drop(index=bike_share[bike_share['ride_length_mins'] < 0].index, inplace=True)

In [10]:
print(bike_share[bike_share['ride_length_mins'] < 0])

Empty DataFrame
Columns: [ride_id, rideable_type, start_time, end_time, start_station_name, start_station_id, end_station_name, end_station_id, start_lat, start_lng, end_lat, end_lng, member_casual, ride_length_mins]
Index: []


In [11]:
#change specific columns to categorical data - easier to manipulate
bike_share = bike_share.astype({'rideable_type':'category','member_casual':'category'})

In [12]:
#Create new columns for day and month - will be used later on for groupby analysis
bike_share['day'] = bike_share['start_time'].dt.day_name().str.slice(stop=3)
bike_share['month'] = bike_share['start_time'].dt.month_name().str.slice(stop=3)
bike_share.head()

,ride_id,rideable_type,start_time,end_time,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_length_mins,day,month
0,6C992BD37A98A63F,classic_bike,2021-04-12 18:25:36,2021-04-12 18:56:55,State St & Pearson St,TA1307000061,Southport Ave & Waveland Ave,13235,41.897448,-87.628722,41.948150,-87.663940,member,31.316667,Mon,Apr
3,1887262AD101C604,classic_bike,2021-04-17 09:17:42,2021-04-17 09:42:48,Honore St & Division St,TA1305000034,Southport Ave & Waveland Ave,13235,41.903119,-87.673935,41.948150,-87.663940,member,25.100000,Sat,Apr
5,097E76F3651B1AC1,classic_bike,2021-04-25 18:43:18,2021-04-25 18:43:59,Clinton St & Polk St,15542,Clinton St & Polk St,15542,41.871467,-87.640949,41.871467,-87.640949,casual,0.683333,Sun,Apr
6,53C38EB01E6FA5C4,classic_bike,2021-04-03 16:28:21,2021-04-03 16:29:47,Ashland Ave & 63rd St,16948,Ashland Ave & 63rd St,16948,41.779374,-87.664843,41.779374,-87.664843,casual,1.433333,Sat,Apr
7,D53AC014EFD6E2BA,electric_bike,2021-04-06 16:35:06,2021-04-06 17:00:56,Dorchester Ave & 49th St,KA1503000069,Dorchester Ave & 49th St,KA1503000069,41.805832,-87.592478,41.805803,-87.592662,casual,25.833333,Tue,Apr


In [13]:
#rename days via list and change dtype to category
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

bike_share['day'] = pd.Categorical(bike_share['day'], categories = days)

In [14]:
#rename months via list and change dtype to category
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

bike_share['month'] = pd.Categorical(bike_share['month'], categories = months)

In [15]:
#checking to ensure no duplicate ride_id
unique_id = len(pd.unique(bike_share['ride_id']))

print("# of unique Id: " + str(unique_id))

# of unique Id: 5415152


In [16]:
#change the lat/long data type to float and then round to 4 d.p. (for coordinates 4 d.p is accurate to 11m which is sufficient in our case)
bike_share.astype({'start_lat':'float', 'start_lng':'float', 'end_lat':'float', 'end_lng':'float'})

rnd_col = ['start_lat','start_lng','end_lat','end_lng']
bike_share[rnd_col] = bike_share[rnd_col].round(4)

In [17]:
#Joining latitude and longitude in a single column to provide a start/end location
bike_share['start_lat_long'] = [', '.join(str(x) for x in y) for y in map(tuple, bike_share[['start_lat', 'start_lng']].values)]
bike_share['end_lat_long'] = [', '.join(str(x) for x in y) for y in map(tuple, bike_share[['end_lat', 'end_lng']].values)]

In [18]:
#drop columns we no longer need - cleaner DF
bike_share.drop(columns= ['start_lat', 'start_lng','end_lat', 'end_lng'], inplace=True)

In [19]:
bike_share.head()

,ride_id,rideable_type,start_time,end_time,start_station_name,start_station_id,end_station_name,end_station_id,member_casual,ride_length_mins,day,month,start_lat_long,end_lat_long
0,6C992BD37A98A63F,classic_bike,2021-04-12 18:25:36,2021-04-12 18:56:55,State St & Pearson St,TA1307000061,Southport Ave & Waveland Ave,13235,member,31.316667,Mon,Apr,"41.8974, -87.6287","41.9482, -87.6639"
3,1887262AD101C604,classic_bike,2021-04-17 09:17:42,2021-04-17 09:42:48,Honore St & Division St,TA1305000034,Southport Ave & Waveland Ave,13235,member,25.100000,Sat,Apr,"41.9031, -87.6739","41.9482, -87.6639"
5,097E76F3651B1AC1,classic_bike,2021-04-25 18:43:18,2021-04-25 18:43:59,Clinton St & Polk St,15542,Clinton St & Polk St,15542,casual,0.683333,Sun,Apr,"41.8715, -87.6409","41.8715, -87.6409"
6,53C38EB01E6FA5C4,classic_bike,2021-04-03 16:28:21,2021-04-03 16:29:47,Ashland Ave & 63rd St,16948,Ashland Ave & 63rd St,16948,casual,1.433333,Sat,Apr,"41.7794, -87.6648","41.7794, -87.6648"
7,D53AC014EFD6E2BA,electric_bike,2021-04-06 16:35:06,2021-04-06 17:00:56,Dorchester Ave & 49th St,KA1503000069,Dorchester Ave & 49th St,KA1503000069,casual,25.833333,Tue,Apr,"41.8058, -87.5925","41.8058, -87.5927"


In [20]:
# Geopy can calculate geodesic distance between two points. The geodesic distance is the shortest distance on the surface of an ellipsoidal model of the earth. 
# The default algorithm uses the method is given by Karney (2013) (geodesic); this is accurate to round-off and always converges.
# To allow for it to work for columns, we define distancer and use row to allow each ID's distance to be calculated and then placed into the journey distance column
# Apply does take some time to execute
# Uk units = miles


def distancer(row):
    coords_1 = (row['start_lat_long'])
    coords_2 = (row['end_lat_long'])
    return distance.distance(coords_1, coords_2).miles

bike_share['journey_distance_miles'] = bike_share.apply(distancer, axis=1)

In [21]:
len(pd.unique(bike_share['start_lat_long']))

28420

In [22]:
len(pd.unique(bike_share['end_lat_long']))

71428

In [23]:
bike_share

,ride_id,rideable_type,start_time,end_time,start_station_name,start_station_id,end_station_name,end_station_id,member_casual,ride_length_mins,day,month,start_lat_long,end_lat_long,journey_distance_miles
0,6C992BD37A98A63F,classic_bike,2021-04-12 18:25:36,2021-04-12 18:56:55,State St & Pearson St,TA1307000061,Southport Ave & Waveland Ave,13235,member,31.316667,Mon,Apr,"41.8974, -87.6287","41.9482, -87.6639",3.947680
3,1887262AD101C604,classic_bike,2021-04-17 09:17:42,2021-04-17 09:42:48,Honore St & Division St,TA1305000034,Southport Ave & Waveland Ave,13235,member,25.100000,Sat,Apr,"41.9031, -87.6739","41.9482, -87.6639",3.155043
5,097E76F3651B1AC1,classic_bike,2021-04-25 18:43:18,2021-04-25 18:43:59,Clinton St & Polk St,15542,Clinton St & Polk St,15542,casual,0.683333,Sun,Apr,"41.8715, -87.6409","41.8715, -87.6409",0.000000
6,53C38EB01E6FA5C4,classic_bike,2021-04-03 16:28:21,2021-04-03 16:29:47,Ashland Ave & 63rd St,16948,Ashland Ave & 63rd St,16948,casual,1.433333,Sat,Apr,"41.7794, -87.6648","41.7794, -87.6648",0.000000
7,D53AC014EFD6E2BA,electric_bike,2021-04-06 16:35:06,2021-04-06 17:00:56,Dorchester Ave & 49th St,KA1503000069,Dorchester Ave & 49th St,KA1503000069,casual,25.833333,Tue,Apr,"41.8058, -87.5925","41.8058, -87.5927",0.010327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5723525,1CEE41690C73108B,classic_bike,2022-03-21 15:12:17,2022-03-21 15:23:01,Michigan Ave & Oak St,13042,Streeter Dr & Grand Ave,13022,member,10.733333,Mon,Mar,"41.901, -87.6238","41.8923, -87.612",0.854842
5723526,A2A6F07D3DF4D0D6,electric_bike,2022-03-09 20:29:48,2022-03-09 21:01:30,Sheridan Rd & Irving Park Rd,13063,Streeter Dr & Grand Ave,13022,casual,31.700000,Wed,Mar,"41.9543, -87.6544","41.8923, -87.612",4.804815
5723529,9C4CE6CC19F8225B,electric_bike,2022-03-09 15:55:26,2022-03-09 16:08:54,NaN,NaN,Albany Ave & Montrose Ave,15621,member,13.466667,Wed,Mar,"41.94, -87.71","41.961, -87.7059",1.464672
5723530,F4E136DEF696F3AE,electric_bike,2022-03-21 16:12:44,2022-03-21 16:18:24,NaN,NaN,Larrabee St & Division St,KA1504000079,member,5.666667,Mon,Mar,"41.91, -87.65","41.9035, -87.6434",0.563057
